In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import sys
import pandas
import numpy as np
import tensorflow as tf
import random

import rtdl
sys.modules['rtdl.rtdl'] = rtdl

In [ ]:
import pickle
data_to_save = pickle.load(open("real_background.pkl", "rb"))

background_x = data_to_save['background_x']
background_y_lbl = data_to_save['background_y_lbl']
pad_in = data_to_save['pad_in']
wrapper_df = data_to_save['wrapper_df']
background_sizes = data_to_save['background_sizes']
col_names = data_to_save['col_names']
to_explain_shape = data_to_save['to_explain_shape']
test_comb = data_to_save['test_comb']

In [ ]:
from explain_icu import TheWrapper

In [ ]:
background_sizes = [x.shape for x in background_x]
model = TheWrapper(wrapper_df, background_sizes, col_names, to_explain_shape)

In [ ]:
pred_vars = wrapper_df['pred_vars']

In [ ]:
pred_var_locs = [i for i in range(len(col_names)) if col_names[i] in pred_vars]

In [ ]:
import sys
sys.path.append(".")
sys.path.append("../required_libs")
sys.path.append("../required_libs/TSInterpret")
sys.path.append("../required_libs/TSInterpret/InterpretabilityModels")
sys.path.append("../required_libs/TSInterpret/InterpretabilityModels/Saliency")

from ex_methods import gen_explanations
import pathlib

In [ ]:
from ex_methods import do_Anchors, do_WindowSHAP, do_COMTE
def get_implemented_methods():
    #List of the methods currently implemented for use
    impl_dict = {
         "CoMTE": {"function": do_COMTE, "result_store": {"explanations": [], "time_taken": [], "random_seed": [], "item_index": [], "samples_explained": []}},
    }
    return impl_dict

In [ ]:
#GENERATE THE EXPLANATIONS

from required_libs.anchor import anchor_tabular

from required_libs.WindowSHAP.windowshap import StationaryWindowSHAP

from required_libs.TSInterpret.InterpretabilityModels.counterfactual.COMTECF import COMTECF

available_exp_methods = get_implemented_methods()

#Select which of the methods to use based on the configuration
methods_to_use = {m:v for m, v in available_exp_methods.items()}


In [ ]:
import pandas
temp = pandas.read_pickle("new_input2.pkl")

In [ ]:
temp['wrapper_df']['combined_dat']['encounter_id'].unique()

In [ ]:
id_to_int_map = {k: int(k+10000000) for k in temp['wrapper_df']['combined_dat']['encounter_id'].unique()}

In [ ]:
def find_records(all_data, list_encids):
    return all_data[all_data['encounter_id'].isin(list_encids)]

In [ ]:
BASE_FOLDER_NAME = 'trial24_vis_outs'
OUTPUT_FOLDERS = ['outs0', 'outs1']#
OUTPUT_FOLDERS = [BASE_FOLDER_NAME + "/" + x for x in OUTPUT_FOLDERS]

In [ ]:
# using only feature names corresponding to features used by the current version of the model
col_names_57 = col_names[1:58]
pred_var_locs = [x-1 for x in pred_var_locs]

In [ ]:
# filtering background to only seqeunces that are at least as long
longest_seq = 28
N_BACKGROUND = -500

front_pad_amt = 48-longest_seq-1
temp_background = background_x[:, -28:, :]
temp_background.shape

bckg_longer_than_seqs = temp_background[temp_background[:, 0, :].sum(axis=-1) > 0]
bckg_longer_than_seqs = bckg_longer_than_seqs[N_BACKGROUND:, :, :]
background_y_labels = model(bckg_longer_than_seqs).argmax(axis=1)

In [ ]:
#code to generate the explanation under different random seeds

for r_seed, outfolder in enumerate(OUTPUT_FOLDERS):
    r_seed = r_seed + 9 #9999
    tf.random.set_seed(r_seed)
    random.seed(r_seed)
    np.random.seed(r_seed)
    
    collected_test_recs = []
    for test_encid in id_to_int_map.keys():
        rec = find_records(temp['wrapper_df']['combined_dat'], [test_encid])
        rec['encounter_id'] = id_to_int_map[test_encid]


        model.to_explain_shape = rec.shape

        rec = np.expand_dims(rec.to_numpy(), axis=0)
        print(f"Starting... {rec.shape}")

        x_in = rec
        y_in = [1] #Using positive cases only

        x_in_length = x_in.shape[1]
        background_x_truncated = bckg_longer_than_seqs[:, -x_in_length:, :]

        explanation_config = {
        'output_folder': outfolder,
        'model': model,
        'feature_names': pred_vars,
        'model_type': 'lstm',
        'what_is_second_dim': 'time',
        'all_names':col_names_57,
         'pred_var_locs': pred_var_locs,
        'background_data': {
            'x': background_x_truncated,
            'y': background_y_labels,
            },
        'experiment_config':{
            'device': 'cpu',
            'categorical_features':[],
            },
        'comte':{
            'max_n_feats': 4,
            'tau_confidence': 0.95,
            'num_background_used': 200,
            'max_attempts': 1000,
            'max_iter': 1000,
            },
        'windowshap':{
            'window_len': 2,
            'num_background_used': 200
            },
        'anchors':{
            'tau': 0.8,
            'delta': 0.1,
            'num_background_used': 200
            }
        }

        gen_explanations(explanation_config, methods_to_use, x_in, y_in)